In [32]:
import pandas as pd
import numpy as np
import glob
from glob import iglob
import os

from dateutil import parser
from datetime import datetime
from dateutil.relativedelta import relativedelta

import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
from dateutil import parser
import sqlalchemy
from datetime import datetime

In [33]:
df = pd.read_csv("moneycontrol2012till2018.csv")
df

,Unnamed: 0,Call_Buy-Sell,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time,Call_Time_normal,Call_Time_secs,Stop_Loss,From_Site
0,1,Buy,Cox and Kings,220.0,ICICI Direct,"November 23, 2018 03:06 PM",2018-11-23 03:06:00,"November 23, 2018 03:06 PM",NaN,MoneyControl
1,2,Buy,IG Petrochemicals,646.0,Nalanda Securities,"November 23, 2018 03:05 PM",2018-11-23 03:05:00,"November 23, 2018 03:05 PM",NaN,MoneyControl
2,3,Buy,Siyaram Silk Mills,460.0,ICICI Direct,"November 23, 2018 03:03 PM",2018-11-23 03:03:00,"November 23, 2018 03:03 PM",NaN,MoneyControl
3,4,Buy,JK Lakshmi Cement,315.0,ICICI Direct,"November 23, 2018 03:02 PM",2018-11-23 03:02:00,"November 23, 2018 03:02 PM",NaN,MoneyControl
4,5,Buy,Sunteck Realty,440.0,ICICI Direct,"November 23, 2018 02:54 PM",2018-11-23 02:54:00,"November 23, 2018 02:54 PM",NaN,MoneyControl
...,...,...,...,...,...,...,...,...,...,...
9069,9077,Buy,Maruti Suzuki India,1668.0,Motilal Oswal,"September 18, 2012 12:09 PM",2012-09-18 12:09:00,"September 18, 2012 12:09 PM",NaN,MoneyControl
9070,9078,Buy,Crompton Greaves,161.0,Microsec,"September 15, 2012 12:15 PM",2012-09-15 12:15:00,"September 15, 2012 12:15 PM",NaN,MoneyControl
9071,9079,Buy,Asian Paints,4250.0,Nirmal Bang,"September 14, 2012 02:36 PM",2012-09-14 02:36:00,"September 14, 2012 02:36 PM",NaN,MoneyControl
9072,9080,Buy,Eros International,188.0,Firstcall,"September 14, 2012 08:22 AM",2012-09-14 08:22:00,"September 14, 2012 08:22 AM",NaN,MoneyControl


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9074 entries, 0 to 9073
Data columns (total 10 columns):
Unnamed: 0          9074 non-null int64
Call_Buy-Sell       9074 non-null object
Cmp_Name            9074 non-null object
Target_Price        9074 non-null float64
Analyst_or_Cmp      9073 non-null object
Call_Time           9074 non-null object
Call_Time_normal    9074 non-null object
Call_Time_secs      9074 non-null object
Stop_Loss           0 non-null float64
From_Site           9074 non-null object
dtypes: float64(2), int64(1), object(7)
memory usage: 709.0+ KB


In [35]:
del df['Unnamed: 0']
del df['Call_Time']
del df['Call_Time_secs']
del df['Stop_Loss']
del df['From_Site']
df

,Call_Buy-Sell,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Buy,Cox and Kings,220.0,ICICI Direct,2018-11-23 03:06:00
1,Buy,IG Petrochemicals,646.0,Nalanda Securities,2018-11-23 03:05:00
2,Buy,Siyaram Silk Mills,460.0,ICICI Direct,2018-11-23 03:03:00
3,Buy,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23 03:02:00
4,Buy,Sunteck Realty,440.0,ICICI Direct,2018-11-23 02:54:00
...,...,...,...,...,...
9069,Buy,Maruti Suzuki India,1668.0,Motilal Oswal,2012-09-18 12:09:00
9070,Buy,Crompton Greaves,161.0,Microsec,2012-09-15 12:15:00
9071,Buy,Asian Paints,4250.0,Nirmal Bang,2012-09-14 02:36:00
9072,Buy,Eros International,188.0,Firstcall,2012-09-14 08:22:00


In [36]:
database_username = 'smarsproddbuser@mbt-smars-mysql'
database_password = 'modern@1234'
database_ip       = 'mbt-smars-mysql.mysql.database.azure.com'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [37]:
df.drop_duplicates(inplace = True)
df

,Call_Buy-Sell,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Buy,Cox and Kings,220.0,ICICI Direct,2018-11-23 03:06:00
1,Buy,IG Petrochemicals,646.0,Nalanda Securities,2018-11-23 03:05:00
2,Buy,Siyaram Silk Mills,460.0,ICICI Direct,2018-11-23 03:03:00
3,Buy,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23 03:02:00
4,Buy,Sunteck Realty,440.0,ICICI Direct,2018-11-23 02:54:00
...,...,...,...,...,...
9069,Buy,Maruti Suzuki India,1668.0,Motilal Oswal,2012-09-18 12:09:00
9070,Buy,Crompton Greaves,161.0,Microsec,2012-09-15 12:15:00
9071,Buy,Asian Paints,4250.0,Nirmal Bang,2012-09-14 02:36:00
9072,Buy,Eros International,188.0,Firstcall,2012-09-14 08:22:00


In [38]:
df1 = df.loc[(df['Call_Buy-Sell'] == 'Buy')]
df1

,Call_Buy-Sell,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Buy,Cox and Kings,220.0,ICICI Direct,2018-11-23 03:06:00
1,Buy,IG Petrochemicals,646.0,Nalanda Securities,2018-11-23 03:05:00
2,Buy,Siyaram Silk Mills,460.0,ICICI Direct,2018-11-23 03:03:00
3,Buy,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23 03:02:00
4,Buy,Sunteck Realty,440.0,ICICI Direct,2018-11-23 02:54:00
...,...,...,...,...,...
9069,Buy,Maruti Suzuki India,1668.0,Motilal Oswal,2012-09-18 12:09:00
9070,Buy,Crompton Greaves,161.0,Microsec,2012-09-15 12:15:00
9071,Buy,Asian Paints,4250.0,Nirmal Bang,2012-09-14 02:36:00
9072,Buy,Eros International,188.0,Firstcall,2012-09-14 08:22:00


In [39]:
df1['Call_Time_normal'] =  pd.to_datetime(df1['Call_Time_normal'], format='%Y-%m-%d %X')
df1

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Call_Buy-Sell,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Buy,Cox and Kings,220.0,ICICI Direct,2018-11-23 03:06:00
1,Buy,IG Petrochemicals,646.0,Nalanda Securities,2018-11-23 03:05:00
2,Buy,Siyaram Silk Mills,460.0,ICICI Direct,2018-11-23 03:03:00
3,Buy,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23 03:02:00
4,Buy,Sunteck Realty,440.0,ICICI Direct,2018-11-23 02:54:00
...,...,...,...,...,...
9069,Buy,Maruti Suzuki India,1668.0,Motilal Oswal,2012-09-18 12:09:00
9070,Buy,Crompton Greaves,161.0,Microsec,2012-09-15 12:15:00
9071,Buy,Asian Paints,4250.0,Nirmal Bang,2012-09-14 02:36:00
9072,Buy,Eros International,188.0,Firstcall,2012-09-14 08:22:00


In [40]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8597 entries, 0 to 9073
Data columns (total 5 columns):
Call_Buy-Sell       8597 non-null object
Cmp_Name            8597 non-null object
Target_Price        8597 non-null float64
Analyst_or_Cmp      8596 non-null object
Call_Time_normal    8597 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 403.0+ KB


In [41]:
start_date = '2016-01-01'
end_date = '2018-12-31'
date_object1 = datetime.strptime(start_date, '%Y-%m-%d').date()
date_object2 = datetime.strptime(end_date, '%Y-%m-%d').date()

In [42]:
mask = (df1['Call_Time_normal'] > date_object1) & (df1['Call_Time_normal'] <= date_object2)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and a TypeError will be raised. To retain the current
behavior, convert the 'datetime.date' to a datetime with
'pd.Timestamp'.
  """Entry point for launching an IPython kernel.


In [43]:
df2 = df1.loc[mask]
df2 = df2.reset_index()
#del df1['level_0']
#del df1['srno']
del df2['index']
df2

,Call_Buy-Sell,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Buy,Cox and Kings,220.0,ICICI Direct,2018-11-23 03:06:00
1,Buy,IG Petrochemicals,646.0,Nalanda Securities,2018-11-23 03:05:00
2,Buy,Siyaram Silk Mills,460.0,ICICI Direct,2018-11-23 03:03:00
3,Buy,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23 03:02:00
4,Buy,Sunteck Realty,440.0,ICICI Direct,2018-11-23 02:54:00
...,...,...,...,...,...
5210,Buy,Samkrg Pistons,210.0,Firstcall,2016-01-05 10:39:00
5211,Buy,HDIL,83.0,Rajat Bose,2016-01-04 10:33:00
5212,Buy,DLF,125.5,Rajat Bose,2016-01-04 09:22:00
5213,Buy,Vinati Organics,555.0,Karvy,2016-01-01 12:37:00


In [44]:
del df2['Call_Buy-Sell']
df2

,Cmp_Name,Target_Price,Analyst_or_Cmp,Call_Time_normal
0,Cox and Kings,220.0,ICICI Direct,2018-11-23 03:06:00
1,IG Petrochemicals,646.0,Nalanda Securities,2018-11-23 03:05:00
2,Siyaram Silk Mills,460.0,ICICI Direct,2018-11-23 03:03:00
3,JK Lakshmi Cement,315.0,ICICI Direct,2018-11-23 03:02:00
4,Sunteck Realty,440.0,ICICI Direct,2018-11-23 02:54:00
...,...,...,...,...
5210,Samkrg Pistons,210.0,Firstcall,2016-01-05 10:39:00
5211,HDIL,83.0,Rajat Bose,2016-01-04 10:33:00
5212,DLF,125.5,Rajat Bose,2016-01-04 09:22:00
5213,Vinati Organics,555.0,Karvy,2016-01-01 12:37:00


In [45]:
df2.to_sql(con=database_connection, name='mcrecos', if_exists='replace')

In [46]:
print("No of Brokers :",df2['Analyst_or_Cmp'].nunique()) 

No of Brokers : 184
